In [0]:
%cd /content/drive/"My Drive"/textgen


/content/drive/My Drive/textgen


In [0]:
%tensorflow_version 1.x
from keras.layers import LSTM, Dense
from keras.utils.data_utils import get_file
from keras.optimizers import RMSprop
from keras.models import Sequential
import numpy as np
import io

Using TensorFlow backend.


In [0]:
path = get_file("shakespear.txt", 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

    # 'nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')


1122304/1115394 [==============================] - 0s 0us/step


In [0]:
with io.open(path, encoding='utf-8') as f: 
  text = f.read().lower()

print("Lenght of text: {} characters.".format(len(text)))
print(text[:300])

Lenght of text: 1115394 characters.
first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you are all resolved rather to die than to famish?

all:
resolved. resolved.

first citizen:
first, you know caius marcius is chief enemy to the people.

all:
we know't, we know't.

first citizen:
let us


In [0]:
chars = sorted(list(set(text)))
print('{} unique chars'.format(len(chars)))
print(chars)

39 unique chars
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [0]:
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

text_as_int = np.array([char_index[c] for c in text])

print("{} ------> {}".format(repr(text[:20]), text_as_int[:20]))

'first citizen:\nbefor' ------> [18 21 30 31 32  1 15 21 32 21 38 17 26 10  0 14 17 18 27 30]


In [0]:
max_len = 128
step = 5
sentences = []
next_chars = []
for i in range(0, len(text) - max_len, step):
  sentences.append(text[i: i + max_len])
  next_chars.append(text[i + max_len])

print('Number of sentences: ', len(sentences))

Number of sentences:  223054


In [0]:
x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_index[char]] = 1
  y[i, char_index[next_chars[i]]] = 1

In [0]:
print(x)
print(y)

[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False  True False ... False False False]]

 [[False  True False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False  True]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False Fal

In [0]:
# build model
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [0]:
def sample(preds, temperature):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(pred) / temperature
  exp_preds = np.exp(preds)

  preds = exp_preds / np.sum(exp_preds)

  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)


In [0]:
import random 
def on_epoch_end(epoch, _):
  textfile = open('epochOutput.txt', 'a')
  print("--------Genrating text after epoch: {} ".format(epoch), file=texfile )

  start_index = rando.randint(0, len(text) - max_len - 1)

  for temp in [0.2, 0.5, 1, 1.2]:
    print("-------Temperature:", temp, file=textfile)

    sentence = text[start_index: start_index + max_len]
    generated = ''
    generated += sentence

    print("------Generated with sentence: '", sentence + "'", file=textfile)


    for i in range(400):
      x_pred = np.zeros((1, max_len, len(char)))
      for t, char in enumerate(sentence):
        x_pred[0, t, char_index[char]] = 1

      preds = model.predict(x_pred)[0]
      next_index = sample(preds, temp)
      next_char = index_char[next_index]

      sentence = sentence[1:] + next_char

      print(next_char, file=textfile, end='')
    print('\n', file=textfile)
  print('\n' + '='*30, file=textfile)

In [0]:
from keras.callbacks import LambdaCallback

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs = 30,
          callbacks=[print_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/30





 20864/223054 [=>............................] - ETA: 7:13 - loss: 2.6507

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-dbeb25711291>", line 8, in <module>
    callbacks=[print_callback])
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1178, in fit
    validation_freq=validation_freq)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_arrays.py", line 204, in fit_loop
    outs = fit_function(ins_batch)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2979, in __call__
    return self._call(inputs)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2937, in _call
    fetched = self._callable_fn(*array_vals)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py", line 1472, in __call__
    run_metadata_ptr)
KeyboardInterrupt

Dur

KeyboardInterrupt: ignored